In [1]:
from lxml import etree
import requests

url = 'http://bang.dangdang.com/books/fivestars/01.00.00.00.00.00-all-0-0-1-1'
data = requests.get(url).text
s = etree.HTML(data)

title = s.xpath('//*[@id="sortRanking"]/div[2]/a/text()')
href = s.xpath('//*[@id="sortRanking"]/div[2]/a/@href')

print(title)
print(href)

['童书']
['http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-1']


In [ ]:
from lxml import etree
import requests

url = 'http://bang.dangdang.com/books/fivestars/01.00.00.00.00.00-all-0-0-1-1'
data = requests.get(url).text
s = etree.HTML(data)
items = s.xpath('//*[@id="sortRanking"]/div')

for item in items:
    book_url=item.xpath('./a/@href')
    item_name=item.xpath('./a/text()')

    if len(book_url)>0:  #避免抓回来的链接是空的情况
        href=book_url[0]
        item_title=item_name[0]
        print(item_title)
        print(href)
        

In [3]:
from lxml import etree
import requests
import time

url = 'http://bang.dangdang.com/books/fivestars/01.00.00.00.00.00-all-0-0-1-1'
data = requests.get(url).text
s = etree.HTML(data)
items = s.xpath('//*[@id="sortRanking"]/div')

for item in items:
    book_url=item.xpath('./a/@href')
    item_name=item.xpath('./a/text()')

    if len(book_url)>0:
        href=book_url[0]
        item_title=item_name[0]
        a=href[41:46]
        print(item_title)
    
        for page in range(1,26):
            per_url= 'http://bang.dangdang.com/books/fivestars/{}.00.00.00.00-all-0-0-1-{}'.format(a,page)
            print(per_url)

童书
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-1
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-2
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-3
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-4
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-5
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-6
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-7
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-8
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-9
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-10
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-11
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-12
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-13
http://bang.dangdang.com/books/fivestars/01.41.00.00.00.00-all-0-0-1-1

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from lxml import etree
import requests
import time

url = 'http://bang.dangdang.com/books/fivestars/01.00.00.00.00.00-all-0-0-1-1'

headers = {
            'Host': 'bang.dangdang.com',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36',
        }

data = requests.get(url,headers=headers).text
s = etree.HTML(data)

with open('dangdang.csv','w') as outputfile:
    items = s.xpath('//*[@id="sortRanking"]/div')

    for item in items:
        book_url=item.xpath('./a/@href')
        item_name=item.xpath('./a/text()')

        if len(book_url)>0:
            href=book_url[0]
            item_title=item_name[0]
            a=href[41:46]
            print(item_title)

            for page in range(1,26):

                per_url= 'http://bang.dangdang.com/books/fivestars/{}.00.00.00.00-all-0-0-1-{}'.format(a,page)
                data2=requests.get(per_url).text
                f=etree.HTML(data2)

                try:
                    file=f.xpath('//ul[@class="bang_list clearfix bang_list_mode"]/li')
                    print('正在打印{}第{}页…………'.format(item_title,page))
                    time.sleep(2)

                    for book in file:
                        title=book.xpath('./div[@class="name"]/a/@title')[0]
                        author=book.xpath('string(./div[@class="publisher_info"][1])')
                        pinglun=book.xpath('./div[@class="star"]/a/text()')[0].strip('条评论')
                        wuxing=book.xpath('./div[@class="biaosheng"]/span/text()')[0].strip('次')
                        price_now=book.xpath('./div[@class="price"]/p/span[1]/text()')[0]
                        price_before=book.xpath('./div[@class="price"]/p/span[2]/text()')[0]
                        price_sale=book.xpath('./div[@class="price"]/p/span[3]/text()')[0]

                        try:
                            date=book.xpath('./div[@class="publisher_info"]/span/text()')[0]
                        except:
                            date='出版时间不详'

                        try:
                            company=book.xpath('./div[@class="publisher_info"][2]/a/text()')[0]
                        except:
                            company='出版社不详'

                        try:
                            price_e=book.xpath('./div[@class="price"]/p[@class="price_e"]/span/text()')[0]
                        except:
                            price_e="没有电子书"

                        outputfile.write('{},{},{},{},{},{},{},{},{},{}'.format(title,author,date,company,pinglun,wuxing,price_now,price_before,price_sale,price_e))

                except:
                    pass

童书
正在打印童书第1页…………
正在打印童书第2页…………
正在打印童书第3页…………
正在打印童书第4页…………
正在打印童书第5页…………
正在打印童书第6页…………
正在打印童书第7页…………
正在打印童书第8页…………
正在打印童书第9页…………
正在打印童书第10页…………
正在打印童书第11页…………
正在打印童书第12页…………
正在打印童书第13页…………
正在打印童书第14页…………
正在打印童书第15页…………
正在打印童书第16页…………
正在打印童书第17页…………
正在打印童书第18页…………
正在打印童书第19页…………
正在打印童书第20页…………
正在打印童书第21页…………
正在打印童书第22页…………
正在打印童书第23页…………
正在打印童书第24页…………
正在打印童书第25页…………
中小学教辅
正在打印中小学教辅第1页…………
正在打印中小学教辅第2页…………
正在打印中小学教辅第3页…………
正在打印中小学教辅第4页…………
正在打印中小学教辅第5页…………
正在打印中小学教辅第6页…………
正在打印中小学教辅第7页…………
正在打印中小学教辅第8页…………
正在打印中小学教辅第9页…………
正在打印中小学教辅第10页…………
正在打印中小学教辅第11页…………
正在打印中小学教辅第12页…………
正在打印中小学教辅第13页…………
正在打印中小学教辅第14页…………
正在打印中小学教辅第15页…………
正在打印中小学教辅第16页…………
正在打印中小学教辅第17页…………
正在打印中小学教辅第18页…………
正在打印中小学教辅第19页…………
正在打印中小学教辅第20页…………
正在打印中小学教辅第21页…………
正在打印中小学教辅第22页…………
正在打印中小学教辅第23页…………
正在打印中小学教辅第24页…………
正在打印中小学教辅第25页…………
小说
正在打印小说第1页…………
正在打印小说第2页…………
正在打印小说第3页…………
正在打印小说第4页…………
正在打印小说第5页…………
正在打印小说第6页…………
正在打印小说第7页…………
正在打印小说第8页…………
正在打印小说第9页…………
正在打印小说第10页…………
正在打印小说第11页…………
正在打印小说第12页…………
正在打印小说第13页

正在打印医学第14页…………
正在打印医学第15页…………
正在打印医学第16页…………
正在打印医学第17页…………
正在打印医学第18页…………
正在打印医学第19页…………
正在打印医学第20页…………
正在打印医学第21页…………
正在打印医学第22页…………
正在打印医学第23页…………
正在打印医学第24页…………
正在打印医学第25页…………
保健/养生
正在打印保健/养生第1页…………
正在打印保健/养生第2页…………
正在打印保健/养生第3页…………
正在打印保健/养生第4页…………
正在打印保健/养生第5页…………
正在打印保健/养生第6页…………
正在打印保健/养生第7页…………
正在打印保健/养生第8页…………
正在打印保健/养生第9页…………
正在打印保健/养生第10页…………
正在打印保健/养生第11页…………
正在打印保健/养生第12页…………
正在打印保健/养生第13页…………
正在打印保健/养生第14页…………
正在打印保健/养生第15页…………
正在打印保健/养生第16页…………
正在打印保健/养生第17页…………
正在打印保健/养生第18页…………
正在打印保健/养生第19页…………
正在打印保健/养生第20页…………
正在打印保健/养生第21页…………
正在打印保健/养生第22页…………
正在打印保健/养生第23页…………
正在打印保健/养生第24页…………
正在打印保健/养生第25页…………
文化
正在打印文化第1页…………
正在打印文化第2页…………
正在打印文化第3页…………
正在打印文化第4页…………
正在打印文化第5页…………
正在打印文化第6页…………
正在打印文化第7页…………
正在打印文化第8页…………
正在打印文化第9页…………
正在打印文化第10页…………
正在打印文化第11页…………
正在打印文化第12页…………
正在打印文化第13页…………
正在打印文化第14页…………
正在打印文化第15页…………
正在打印文化第16页…………
正在打印文化第17页…………
正在打印文化第18页…………
正在打印文化第19页…………
正在打印文化第20页…………
正在打印文化第21页…………
正在打印文化第22页…………
正在打印文化第23页…………
正在打印文化第24页…………
正在打印文化第25页…………
经济
正

正在打印手工/DIY第23页…………
正在打印手工/DIY第24页…………
正在打印手工/DIY第25页…………
休闲/爱好
正在打印休闲/爱好第1页…………
正在打印休闲/爱好第2页…………
正在打印休闲/爱好第3页…………
正在打印休闲/爱好第4页…………
正在打印休闲/爱好第5页…………
正在打印休闲/爱好第6页…………
正在打印休闲/爱好第7页…………
正在打印休闲/爱好第8页…………
正在打印休闲/爱好第9页…………
正在打印休闲/爱好第10页…………
正在打印休闲/爱好第11页…………
正在打印休闲/爱好第12页…………
正在打印休闲/爱好第13页…………
正在打印休闲/爱好第14页…………
正在打印休闲/爱好第15页…………
正在打印休闲/爱好第16页…………
正在打印休闲/爱好第17页…………
正在打印休闲/爱好第18页…………
正在打印休闲/爱好第19页…………
正在打印休闲/爱好第20页…………
正在打印休闲/爱好第21页…………
正在打印休闲/爱好第22页…………
正在打印休闲/爱好第23页…………
正在打印休闲/爱好第24页…………
正在打印休闲/爱好第25页…………
农业/林业
正在打印农业/林业第1页…………
正在打印农业/林业第2页…………
正在打印农业/林业第3页…………
正在打印农业/林业第4页…………
正在打印农业/林业第5页…………
正在打印农业/林业第6页…………
正在打印农业/林业第7页…………
正在打印农业/林业第8页…………
正在打印农业/林业第9页…………
正在打印农业/林业第10页…………
正在打印农业/林业第11页…………
正在打印农业/林业第12页…………
正在打印农业/林业第13页…………
正在打印农业/林业第14页…………
正在打印农业/林业第15页…………
正在打印农业/林业第16页…………
正在打印农业/林业第17页…………
正在打印农业/林业第18页…………
正在打印农业/林业第19页…………
正在打印农业/林业第20页…………
正在打印农业/林业第21页…………
正在打印农业/林业第22页…………
正在打印农业/林业第23页…………
正在打印农业/林业第24页…………
正在打印农业/林业第25页…………
工具书
正在打印工具书第1页…………
正在打印工具书第2页…………
正在打印工具书第3页…………
